# Developing behavioral data preprocessing pipeline for OPM MEG analysis


## Preprocessing Steps incuded:
- Loading, syncing, and alligning data 
- Categorizing ACC and EMG data into rest vs muscular activity vs movement
-


### 0. Importing

In [ ]:
# general packages
import json
import os
import importlib
import sys
import numpy as np
import pandas as pd
import pyxdf
from itertools import compress
import matplotlib.pyplot as plt
import datetime as dt

# ephys packages
# from mne.filter import filter_data, notch_filter

In [ ]:
def add_repo_dir():
    """adds local repo directory to sys to allow importing from repo"""

    wd = os.getcwd()

    COUNTER = 20  #  to prevent eternal while loop

    while not wd.endswith('lid_opm'):
        wd = os.path.dirname(wd)
        COUNTER -= 1

        if COUNTER == 0:
            raise ValueError('repo dir not found!')

    print(f'add repo directory to sys: {wd} ')

    sys.path.append(wd)

    return

In [ ]:
# add custom functions

add_repo_dir()

import utils.load_utils as load_utils
from preprocessing_meg import load_source_opm as source_opm


## 1. Load behavioral source data

Define:
- subject
- task
- configuration version


In [ ]:
CONFIG_VERSION = "v1"

SUB = '03'  # 
TASK = 'task'
ACQ = 'dopa20'


# load settings
sub_config = load_utils.load_subject_config(subject_id=SUB,)
preproc_config = load_utils.load_preproc_config(version=CONFIG_VERSION,)
sub_meta_info = load_utils.get_sub_rec_metainfo(config_sub=sub_config)




In [ ]:
sub_meta_info

In [ ]:
import source_raw_conversion.load_lsl as loadlsl
import source_raw_conversion.time_syncing as sync
import source_raw_conversion.load_source_opm as sourceopm

manual dev of source lsl to raw flow

In [ ]:
importlib.reload(loadlsl)
importlib.reload(sync)


(
    auxdat, auxtimes, aux_chnames,
    aux_sfreq, pyg_timings
) = loadlsl.convert_source_lsl_to_raw(SUB, TASK, ACQ)

In [ ]:
pyg_timings

In [ ]:
print(auxdat.shape, auxtimes.shape)
print(aux_chnames)

In [ ]:
# # check data channeling
# for i, chname in enumerate(sub_config["antneuro_chs"].values()):
#     plt.title(chname)
#     plt.plot(auxdat[:, i])
#     plt.show()

Visualization of imported data

In [ ]:
%matplotlib qt

In [ ]:
sample = game_timings['abort_left']['start']
plt.scatter(sample, np.ones(len(sample)), c='green',)

sample = game_timings['abort_left']['abort']
plt.scatter(sample, np.ones(len(sample)), color='orange')

sample = game_timings['abort_left']['end']
plt.scatter(sample, np.ones(len(sample)), color='gray',)


plt.show()

## 2- Resample and Filter

The following preprocessing steps will be applied to the MEG data to prepare it for analysis:

**Configuration Parameters:**
- Target sampling frequency: 512 Hz
- Bandpass filter: 1-100 Hz  
- Notch filter frequencies: 50, 100, 150 Hz

Resample

In [ ]:
meg_dat = raw.copy()

In [ ]:
# resample
if meg_dat.info['sfreq'] <= preproc_config['TARGET_SFREQ']:
    print(f'original sampling rate {meg_dat.info["sfreq"]} vs defined new rate {preproc_config["TARGET_SFREQ"]}')
else:
    print(f'resample original sampling rate {meg_dat.info["sfreq"]} to {preproc_config["TARGET_SFREQ"]}')
    meg_dat = meg_dat.resample(preproc_config['TARGET_SFREQ'], verbose=False)

# # convert to picoTesla
# meg_dat._data = meg_dat.get_data() * 1e12

Filter

- Bandpass filter
- Notch filter

In [ ]:
temp_dat = meg_dat.copy()

# Bandpass filter (1-100 Hz); use .filter() to remain Raw Mne Object
temp_dat = temp_dat.filter(
    l_freq=preproc_config['BANDPASS_LOW'],
    h_freq=preproc_config['BANDPASS_HIGH'], 
    method='fir', verbose=False,
)  # sfreq=meg_dat.info['sfreq'], is given within Raw Object
meg_dat = temp_dat.copy()

# Apply notch filters (50 Hz and harmonics)
temp_dat = temp_dat.get_data()
for freq in preproc_config['NOTCH_FREQS']:
    temp_dat = notch_filter(
        temp_dat, 
        Fs=meg_dat.info['sfreq'],  
        freqs=freq,
        verbose=False
    )

meg_dat._data = temp_dat

In [ ]:
# fig = meg_dat.plot(start=0, duration=1,)



# fig_dat = meg_dat.get_data()
# N_SEC = 1

# fig, axes = plt.subplots(fig_dat.shape[0])
# for i_ax in np.arange(fig_dat.shape[0]):
    # axes[i_ax].plot(fig_dat[i_ax, :int(N_SEC * meg_dat.info['sfreq'])])

# for ax in axes:
#     ax.

# plt.show()